In [1]:
import os
%pwd

'c:\\Users\\Public\\Documents\\Project\\Kidney-Diseases-Classification\\research'

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\Public\\Documents\\Project\\Kidney-Diseases-Classification'

In [32]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path
    data_dir:Path
    split_dir:Path


In [33]:
from kidney_diseases_classification.constants import *
from kidney_diseases_classification.utils.common import read_yaml,create_directories

In [34]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir, 
            data_dir=config.data_dir,
            split_dir=config.split_dir
        )

        return data_ingestion_config

In [35]:
pip install split-folders

Note: you may need to restart the kernel to use updated packages.


In [36]:
import os
import zipfile
import gdown
from kidney_diseases_classification import logger
from kidney_diseases_classification.utils.common import get_size
import splitfolders

In [37]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
   
    def split_folder(self):
        """
        split_folder_path: str
        split the dataset in train test val
        Function returns None
        """
        split_path=self.config.root_dir
        os.makedirs(split_path, exist_ok=True)
        pa=%pwd
        full_path = os.path.join(pa,self.config.root_dir, self.config.data_dir)
        full_path = Path(full_path)
        splitfolders.ratio(full_path,output=self.config.split_dir,seed=7,ratio=(0.8,0.1, 0.1))




In [38]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    data_ingestion.split_folder()
except Exception as e:
    raise e

[2023-10-18 20:41:41,780: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-18 20:41:41,783: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-18 20:41:41,786: INFO: common: created directory at: artifacts]
[2023-10-18 20:41:41,787: INFO: common: created directory at: artifacts/data_ingestion]


Copying files: 12446 files [00:10, 1167.15 files/s]
